In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
import seaborn as sns
import datetime as dt
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
df = pd.read_csv('data/master_data_with_sequel_director.csv')

In [3]:
df['release_date'] = pd.to_datetime(df['release_date'])
df['year'] = pd.DatetimeIndex(df['release_date']).year

In [4]:
df['returnb'] = df['domestic_gross'] / df['budget']

In [5]:
x = df[['budget','is_sequel','rating','top50_d','year','runtime_minutes','month','dd_top5','bankable_count','Action','Adventure','Animation','Comedy','Drama','Horror','Mystery','Romance']]


y = df['domestic_gross']

x_tr, x_test, y_tr, y_test = train_test_split(x, y,test_size=.25, random_state=10)
#5

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x_tr, y_tr, test_size=.25, random_state=10)
#10

In [7]:
x_cont_train = x_train[['budget','is_sequel','top50_d','year','month','dd_top5','bankable_count','runtime_minutes','Action','Adventure','Animation','Comedy','Drama','Horror']]                        
x_cont_val = x_val[['budget','is_sequel','top50_d','year','month','dd_top5','bankable_count','runtime_minutes','Action','Adventure','Animation','Comedy','Drama','Horror']]
x_cont_test = x_test[['budget','is_sequel','top50_d','year','month','dd_top5','bankable_count','runtime_minutes','Action','Adventure','Animation','Comedy','Drama','Horror']]

In [24]:
x_cont_train['BUD2'] = x_cont_train[['budget']]**2
x_cont_val['BUD2'] = x_cont_val[['budget']]**2
x_cont_test['BUD2'] = x_test[['budget']]**2

<ipython-input-24-d0eed0c2778d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_cont_train['BUD2'] = x_cont_train[['budget']]**2
<ipython-input-24-d0eed0c2778d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_cont_val['BUD2'] = x_cont_val[['budget']]**2
<ipython-input-24-d0eed0c2778d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [9]:
x_cat_train = x_train[['rating']]
x_cat_val = x_val[['rating']]
x_cat_test = x_test[['rating']]

In [28]:
# Dummify rating for train
ohe = OneHotEncoder(drop='first', sparse=False)

ohe.fit(x_cat_train)

ohe_x_train = ohe.transform(x_cat_train)

columns = ohe.get_feature_names(['rating'])

ohe_x_train_df = pd.DataFrame(ohe_x_train, columns=columns, index=x_cat_train.index)

In [27]:
# Dummify rating for val
ohe_v = OneHotEncoder(drop='first', sparse=False)

ohe_v.fit(x_cat_val)

ohe_x_val = ohe_v.transform(x_cat_val)

columns_v = ohe_v.get_feature_names(['rating'])

ohe_x_val_df = pd.DataFrame(ohe_x_val, columns=columns_v, index=x_cat_val.index)

In [25]:
# Dummify rating for test
ohe_te = OneHotEncoder(drop='first', sparse=False)

ohe_te.fit(x_cat_test)

ohe_x_test = ohe_te.transform(x_cat_test)

columns_te = ohe_te.get_feature_names(['rating'])

ohe_x_test_df = pd.DataFrame(ohe_x_test, columns=columns_te, index=x_cat_test.index)

In [26]:
# Combine cat and cont features in single dataframe
combo_train_df = pd.concat([x_cont_train, ohe_x_train_df], axis=1)

combo_val_df =  pd.concat([x_cont_val, ohe_x_val_df], axis=1)

combo_test_df =  pd.concat([x_cont_test, ohe_x_test_df], axis=1)

In [30]:
m = LinearRegression()
p = PolynomialFeatures(degree=2, interaction_only=True)
x_train_poly = p.fit_transform(combo_train_df)
m.fit(x_train_poly,y_train)

print('R^2 for train with poly features:')
m.score(x_train_poly,y_train)

R^2 for train with poly features:


0.47476619455241564

In [15]:
list(zip(p.get_feature_names(['budget','BUD2','is_sequel','top50_d','year','month','dd_top5','bankable_count','runtime_minutes','Action','Adventure','Animation','Comedy','Drama','Horror','R','G','PG','PG-13']),m.coef_))

[('1', 1.7495309939150382e-07),
 ('budget', -3.5165164249591157e-06),
 ('BUD2', 5.029327376694558e-11),
 ('is_sequel', 2.1634789828161183e-13),
 ('top50_d', 1.4279564669027668e-12),
 ('year', 1.6857020945086775e-13),
 ('month', 8.083416319096496e-14),
 ('dd_top5', 4.042509963541738e-14),
 ('bankable_count', -3.0492988464131826e-14),
 ('runtime_minutes', 2.9266462415148912e-15),
 ('Action', 9.767031637585329e-14),
 ('Adventure', -5.3712217594416535e-14),
 ('Animation', 7.201216895824855e-14),
 ('Comedy', -7.416204650841351e-14),
 ('Drama', -3.681193393407111e-13),
 ('Horror', -1.7010415286154878e-07),
 ('R', 3.49961669066153e-12),
 ('G', -4.050973006760545e-17),
 ('PG', -8.328571013955366e-16),
 ('budget BUD2', 1.4264158673208171e-08),
 ('budget is_sequel', -4.838145148351218e-09),
 ('budget top50_d', 0.00024371294977900454),
 ('budget year', 1.0172761402958627e-06),
 ('budget month', 4.3320659043522024e-08),
 ('budget dd_top5', -4.7244117726674764e-08),
 ('budget bankable_count', 9.891

In [31]:
print('R^2 for val with poly features:')
x_val_poly = p.fit_transform(combo_val_df)
m.score(x_val_poly,y_val)

R^2 for val with poly features:


0.17701117299979596

In [32]:
print('R^2 for test with poly features:')
x_test_poly = p.fit_transform(combo_test_df)
m.score(x_test_poly,y_test)

R^2 for test with poly features:


0.4006856161074469

Wide variation in R^2 across samples. Data in val sample must be very different to train and test. Need to reshuffle and use Lasso to whiddle down variables as this is overfitting.

In [33]:
from sklearn.linear_model import LassoCV

In [34]:
lcv = LassoCV()
p = PolynomialFeatures(degree=2,interaction_only=True)
x_train_poly = p.fit_transform(combo_train_df)
s = StandardScaler(with_mean=False)
x_train_poly_scaled = s.fit_transform(x_train_poly)
lcv.fit(x_train_poly_scaled,y_train)

print('R^2 for train with poly features (lasso):')
lcv.score(x_train_poly_scaled,y_train)

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 495631183671552.0, tolerance: 446361448024842.2
  model = cd_fast.enet_coordinate_descent_gram(
/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 604556533480960.0, tolerance: 446361448024842.2
  model = cd_fast.enet_coordinate_descent_gram(
/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 948051111483136.0, tolerance: 446361448024842.2
  model = cd_fast.enet_coordinate_descent_gram(
/Users/tawneykirkland/opt/anaco

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 566531083976192.0, tolerance: 429396733977507.4
  model = cd_fast.enet_coordinate_descent_gram(
/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 879477705859072.0, tolerance: 429396733977507.4
  model = cd_fast.enet_coordinate_descent_gram(
/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1517232313482752.0, tolerance: 429396733977507.4
  model = cd_fast.enet_coordinate_descent_gram(
/Users/tawneykirkland/opt/anac

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2454294554447616.0, tolerance: 442204418692040.1
  model = cd_fast.enet_coordinate_descent_gram(
/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1974579203208448.0, tolerance: 442204418692040.1
  model = cd_fast.enet_coordinate_descent_gram(
/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6716708785106432.0, tolerance: 442204418692040.1
  model = cd_fast.enet_coordinate_descent_gram(
/Users/tawneykirkland/opt/an

R^2 for train with poly features (lasso):


/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6203757186761728.0, tolerance: 414210185112783.5
  model = cd_fast.enet_coordinate_descent_gram(
/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9294499170859520.0, tolerance: 414210185112783.5
  model = cd_fast.enet_coordinate_descent_gram(
/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.2820449058941952e+16, tolerance: 414210185112783.5
  model = cd_fast.enet_coordinate_descent_gram(


0.48085547220549785

In [35]:
print('R^2 for val with poly features (lasso):')
lcv.score(s.transform(p.transform(combo_val_df)),y_val)

R^2 for val with poly features (lasso):


0.23617893679295543

In [36]:
print('R^2 for test with poly features (lasso):')
lcv.score(s.transform(p.transform(combo_test_df)),y_test)

R^2 for test with poly features (lasso):


0.4176094136357055

Same issue as above. Reshuffle in main notebook.

In [37]:
print('Lasso alpha:')
lcv.alpha_

Lasso alpha:


5094531.172025066

In [38]:
list(zip(p.get_feature_names(['budget','BUD2','is_sequel','top50_d','year','month',
                              'dd_top5','bankable_count','runtime_minutes','Action',
                              'Adventure','Animation','Comedy','Drama','Horror','R',
                              'G','PG','PG-13']),lcv.coef_))

[('1', 0.0),
 ('budget', 0.0),
 ('BUD2', 0.0),
 ('is_sequel', 0.0),
 ('top50_d', 0.0),
 ('year', -0.0),
 ('month', -0.0),
 ('dd_top5', 0.0),
 ('bankable_count', 0.0),
 ('runtime_minutes', 0.0),
 ('Action', 0.0),
 ('Adventure', 0.0),
 ('Animation', 0.0),
 ('Comedy', -0.0),
 ('Drama', -0.0),
 ('Horror', 0.0),
 ('R', 0.0),
 ('G', 0.0),
 ('PG', -0.0),
 ('budget BUD2', 0.0),
 ('budget is_sequel', 0.0),
 ('budget top50_d', 0.0),
 ('budget year', 0.0),
 ('budget month', 0.0),
 ('budget dd_top5', 0.0),
 ('budget bankable_count', 0.0),
 ('budget runtime_minutes', 0.0),
 ('budget Action', 0.0),
 ('budget Adventure', 0.0),
 ('budget Animation', 0.0),
 ('budget Comedy', -0.0),
 ('budget Drama', -0.0),
 ('budget Horror', 0.0),
 ('budget R', -0.0),
 ('budget G', 0.0),
 ('budget PG', -0.0),
 ('BUD2 is_sequel', 9654119.250980048),
 ('BUD2 top50_d', 0.0),
 ('BUD2 year', 0.0),
 ('BUD2 month', -0.0),
 ('BUD2 dd_top5', 0.0),
 ('BUD2 bankable_count', 0.0),
 ('BUD2 runtime_minutes', 0.0),
 ('BUD2 Action', 0

Can begin removing the zeroed out ones

Also remove the ones that do not make sense to ensure interpretability.